# Data Cleaning 

#### 1. Import pandas library.

In [1]:
import pandas as pd

#### 2. Import pymysql and sqlalchemy as you have learnt in the lesson of importing/exporting data. 


#### 3. Create a mysql engine to set the connection to the server. Check the connection details in [this link](https://relational.fit.cvut.cz/dataset/Stats).

#### 4. Import the users table.

In [2]:
users = pd.read_csv("../users.csv")
users.drop("Unnamed: 0", axis=1)

,userId,Reputation,Views,UpVotes,DownVotes
0,-1,1,0,5007,1920
1,2,101,25,3,0
2,3,101,22,19,0
3,4,101,11,0,0
4,5,6792,1145,662,5
...,...,...,...,...,...
40320,55743,1,0,0,0
40321,55744,6,1,0,0
40322,55745,101,0,0,0
40323,55746,106,1,0,0


#### 5. Rename Id column to userId.

In [3]:
# Already done

#### 6. Import the posts table. 

In [4]:
posts = pd.read_csv("../posts.csv")
posts.drop('Unnamed: 0', axis=1)
posts

,Unnamed: 0,PostId,userId,Score,ViewCount,CommentCount
0,0,1,8.0,23,1278.0,1
1,1,2,24.0,22,8198.0,1
2,2,3,18.0,54,3613.0,4
3,3,4,23.0,13,5224.0,2
4,4,5,23.0,81,NaN,3
...,...,...,...,...,...,...
91971,91971,115374,805.0,2,NaN,2
91972,91972,115375,49365.0,0,9.0,0
91973,91973,115376,55746.0,1,5.0,2
91974,91974,115377,805.0,0,NaN,0


#### 7. Rename Id column to postId and OwnerUserId to userId.

In [5]:
# Already done

#### 8. Define new dataframes for users and posts with the following selected columns:
**users_sliced columns**: userId, Reputation, Views, UpVotes, DownVotes  
**posts_sliced columns**: postId, Score, userId, ViewCount, CommentCount

In [6]:
users_sliced=users[["userId", "Reputation", "Views", "UpVotes", "DownVotes"]]
posts_sliced=posts[["PostId", "Score", "userId", "ViewCount", "CommentCount"]]

#### 9. Merge the two dataframes created in the step above (8), users_sliced and posts_sliced. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [7]:
users_posts = users_sliced.merge(posts_sliced, how = "inner", on = "userId")
users_posts

,userId,Reputation,Views,UpVotes,DownVotes,PostId,Score,ViewCount,CommentCount
0,-1,1,0,5007,1920,2175,0,NaN,0
1,-1,1,0,5007,1920,8576,0,NaN,0
2,-1,1,0,5007,1920,8578,0,NaN,0
3,-1,1,0,5007,1920,8981,0,NaN,0
4,-1,1,0,5007,1920,8982,0,NaN,0
...,...,...,...,...,...,...,...,...,...
90579,55734,1,0,0,0,115352,0,16.0,0
90580,55738,11,0,0,0,115360,2,40.0,4
90581,55742,6,0,0,0,115366,1,17.0,0
90582,55744,6,1,0,0,115370,1,13.0,2


#### 10. How many missing values do you have in your merged dataframe? On which columns?

In [8]:
users_posts.isnull().sum()

userId              0
Reputation          0
Views               0
UpVotes             0
DownVotes           0
PostId              0
Score               0
ViewCount       48396
CommentCount        0
dtype: int64

#### 11. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before going to the next step.

In [9]:
users_posts = users_posts.fillna(0)

#### 12. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [10]:
# Change ViewCount from float ot integer
users_posts = users_posts.astype(int)
users_posts.dtypes

userId          int64
Reputation      int64
Views           int64
UpVotes         int64
DownVotes       int64
PostId          int64
Score           int64
ViewCount       int64
CommentCount    int64
dtype: object

In [11]:
users_posts.corr()

,userId,Reputation,Views,UpVotes,DownVotes,PostId,Score,ViewCount,CommentCount
userId,1.000000,-0.344814,-0.301128,-0.293232,-0.199850,0.704867,-0.233026,-0.071242,-0.032604
Reputation,-0.344814,1.000000,0.906704,0.852445,0.558412,-0.076308,0.124807,-0.054259,0.041730
Views,-0.301128,0.906704,1.000000,0.805355,0.636178,-0.122016,0.128674,-0.046159,0.049023
UpVotes,-0.293232,0.852445,0.805355,1.000000,0.636087,-0.099961,0.130299,-0.046696,0.028820
DownVotes,-0.199850,0.558412,0.636178,0.636087,1.000000,-0.079508,0.073268,-0.034530,0.002877
PostId,0.704867,-0.076308,-0.122016,-0.099961,-0.079508,1.000000,-0.262381,-0.129319,-0.041256
Score,-0.233026,0.124807,0.128674,0.130299,0.073268,-0.262381,1.000000,0.333298,0.148255
ViewCount,-0.071242,-0.054259,-0.046159,-0.046696,-0.034530,-0.129319,0.333298,1.000000,0.044059
CommentCount,-0.032604,0.041730,0.049023,0.028820,0.002877,-0.041256,0.148255,0.044059,1.000000


In [12]:
users_new = users_posts[users_posts["ViewCount"] != 0]

In [13]:
users_new.corr()

,userId,Reputation,Views,UpVotes,DownVotes,PostId,Score,ViewCount,CommentCount
userId,1.000000,-0.208943,-0.199015,-0.167294,-0.118117,0.801847,-0.263157,-0.181916,-0.057207
Reputation,-0.208943,1.000000,0.915478,0.731704,0.613213,-0.127497,0.195421,0.057293,0.036205
Views,-0.199015,0.915478,1.000000,0.698065,0.640884,-0.130170,0.177490,0.056347,0.038557
UpVotes,-0.167294,0.731704,0.698065,1.000000,0.586170,-0.117692,0.172966,0.046272,0.025198
DownVotes,-0.118117,0.613213,0.640884,0.586170,1.000000,-0.085031,0.132649,0.033729,0.032103
PostId,0.801847,-0.127497,-0.130170,-0.117692,-0.085031,1.000000,-0.303660,-0.235046,-0.068333
Score,-0.263157,0.195421,0.177490,0.172966,0.132649,-0.303660,1.000000,0.532106,0.110266
ViewCount,-0.181916,0.057293,0.056347,0.046272,0.033729,-0.235046,0.532106,1.000000,0.044713
CommentCount,-0.057207,0.036205,0.038557,0.025198,0.032103,-0.068333,0.110266,0.044713,1.000000
